In [8]:
using DifferentiableStateSpaceModels, DifferenceEquations, LinearAlgebra, Turing, Zygote
using DifferentiableStateSpaceModels.Examples
using Turing: @addlogprob!

Turing.setadbackend(:zygote)

# Create models from modules and then solve
model_rbc = @include_example_module(Examples.rbc_observables)

# Generate artificial data for estimation
p_f = (ρ = 0.2, δ = 0.02, σ = 0.01, Ω_1 = 0.01) # Fixed parameters
p_d = (α = 0.5, β = 0.95) # Pseudo-true values
sol = generate_perturbation(model_rbc, p_d, p_f, Val(1)) # Solution to the first-order RBC
sol_second = generate_perturbation(model_rbc, p_d, p_f, Val(2)) # Solution to the second-order RBC

T = 20
ϵ = [randn(model_rbc.n_ϵ) for _ in 1:T]
x0 = zeros(model_rbc.n_x)

# Create a first-order problem setting
problem_first_order = StateSpaceProblem(
    DifferentiableStateSpaceModels.dssm_evolution,
    DifferentiableStateSpaceModels.dssm_volatility,
    DifferentiableStateSpaceModels.dssm_observation,
    x0,
    (0, T),
    sol,
    noise = DefinedNoise(ϵ),
)
# Generate fake data for first-order estimation exercises
fake_z = DifferenceEquations.solve(problem_first_order, NoiseConditionalFilter()).z[2:end]

# Create a second-order problem setting
problem_second_order = StateSpaceProblem(
    DifferentiableStateSpaceModels.dssm_evolution,
    DifferentiableStateSpaceModels.dssm_volatility,
    DifferentiableStateSpaceModels.dssm_observation,
    [x0; x0],
    (0, T),
    sol_second,
    noise = DefinedNoise(ϵ),
)
# Generate fake data for second-order estimatino exercises
fake_z_second = DifferenceEquations.solve(problem_second_order, NoiseConditionalFilter()).z[2:end]

20-element Vector{Vector{Float64}}:
 [0.010334871038349262, -7.824904812740593e-5]
 [0.009423566208750341, 0.0943670667134858]
 [0.015008764884793328, 0.09303935790924596]
 [0.015016700100943674, 0.1443767660379526]
 [0.02113467304000077, 0.14829744045734816]
 [0.013036180623260741, 0.20494961851357657]
 [0.020128017797618636, 0.1346337678757667]
 [0.017169363491181408, 0.19465230103200418]
 [0.016957726243991545, 0.17190138588597445]
 [0.006459137337200757, 0.16824052733838765]
 [0.011332069203908293, 0.07134164588489617]
 [0.01427844664817287, 0.1088856414442139]
 [0.0036495297240985917, 0.13883510628593423]
 [-0.0009176958065385961, 0.04326796775724953]
 [0.002087202549504044, -0.005980498232478319]
 [0.008575421729870135, 0.01796423681254636]
 [0.003777787105670909, 0.07950304294265159]
 [-0.008164894519121842, 0.03995580512836877]
 [-0.005196677187614779, -0.0728703298489967]
 [-0.005802045134383482, -0.05409787878535121]

In [9]:
## Estimation example: first-order, marginal likelihood approach

# Turing model definition
@model function rbc_kalman(z, m, p_f, cache)
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (; α, β)
    sol = generate_perturbation(m, p_d, p_f, Val(1); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    problem = LinearStateSpaceProblem(
        sol.A,
        sol.B,
        sol.C,
        sol.x_ergodic,
        (0, T),
        obs_noise = sol.D,
        observables = z
    )
    @addlogprob! DifferenceEquations.solve(problem, KalmanFilter(); vectype = Zygote.Buffer).loglikelihood
end

c = SolverCache(model_rbc, Val(1), p_d)
turing_model = rbc_kalman(fake_z, model_rbc, p_f, c)
n_samples = 1000
n_adapts = 100
δ = 0.65
chain = sample(turing_model, NUTS(n_adapts, δ), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.4
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\nfMhU\src\inference\hmc.jl:188
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\wupei\.julia\packages\AdvancedHMC\HQHnm\src\hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:35


Chains MCMC chain (1000×14×1 Array{Float64, 3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 109.88 seconds
Compute duration  = 109.88 seconds
parameters        = α, β
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4836    0.0313     0.0010    0.0017   264.0063    1.0024     ⋯
           β    0.9514    0.0124     0.0004    0.0008   217.9021    1.0011     ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

       

In [12]:
# Turing model definition
@model function rbc_joint(z, m, p_f, cache::DifferentiableStateSpaceModels.AbstractSolverCache{Order}, x0) where {Order}
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p_d = (; α, β)
    T = length(z)
    ϵ_draw ~ MvNormal(T, 1.0)
    ϵ = map(i -> ϵ_draw[((i-1)*m.n_ϵ+1):(i*m.n_ϵ)], 1:T)
    sol = generate_perturbation(m, p_d, p_f, Val(Order); cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    problem = StateSpaceProblem(
        DifferentiableStateSpaceModels.dssm_evolution,
        DifferentiableStateSpaceModels.dssm_volatility,
        DifferentiableStateSpaceModels.dssm_observation,
        x0,
        (0, T),
        sol,
        noise = DefinedNoise(ϵ),
        obs_noise = sol.D,
        observables = z
    )
    @addlogprob! DifferenceEquations.solve(problem, NoiseConditionalFilter(); vectype = Zygote.Buffer).loglikelihood
end

rbc_joint (generic function with 2 methods)

In [13]:
## Estimation example: first-order, joint likelihood approach
c = SolverCache(model_rbc, Val(1), p_d)
turing_model = rbc_joint(fake_z, model_rbc, p_f, c, x0)
n_samples = 1000
n_adapts = 100
δ = 0.65
max_depth = 5 # A lower max_depth will lead to higher autocorrelation of samples, but faster. The time complexity is approximately 2^max_depth
chain = sample(turing_model, NUTS(n_adapts, δ; max_depth), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\nfMhU\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:06:13


Chains MCMC chain (1000×34×1 Array{Float64, 3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 414.73 seconds
Compute duration  = 414.73 seconds
parameters        = α, β, ϵ_draw[1], ϵ_draw[2], ϵ_draw[3], ϵ_draw[4], ϵ_draw[5], ϵ_draw[6], ϵ_draw[7], ϵ_draw[8], ϵ_draw[9], ϵ_draw[10], ϵ_draw[11], ϵ_draw[12], ϵ_draw[13], ϵ_draw[14], ϵ_draw[15], ϵ_draw[16], ϵ_draw[17], ϵ_draw[18], ϵ_draw[19], ϵ_draw[20]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4889    0.0288     0.0009    0.0036    35.2358    1.0522     ⋯
           β    0.9520    0.0120     0.0004    0.0013    45.5091    1.0589    

In [14]:
## Estimation example: second-order, joint likelihood approach
c = SolverCache(model_rbc, Val(2), p_d)
turing_model = rbc_joint(fake_z_second, model_rbc, p_f, c, [x0; x0])
n_samples = 1000
n_adapts = 100
δ = 0.65
max_depth = 5
chain = sample(turing_model, NUTS(n_adapts, δ; max_depth), n_samples; progress = true)

┌ Info: Found initial step size
│   ϵ = 0.0234375
└ @ Turing.Inference C:\Users\wupei\.julia\packages\Turing\nfMhU\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:11:41


Chains MCMC chain (1000×34×1 Array{Float64, 3}):

Iterations        = 101:1:1100
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 733.25 seconds
Compute duration  = 733.25 seconds
parameters        = α, β, ϵ_draw[1], ϵ_draw[2], ϵ_draw[3], ϵ_draw[4], ϵ_draw[5], ϵ_draw[6], ϵ_draw[7], ϵ_draw[8], ϵ_draw[9], ϵ_draw[10], ϵ_draw[11], ϵ_draw[12], ϵ_draw[13], ϵ_draw[14], ϵ_draw[15], ϵ_draw[16], ϵ_draw[17], ϵ_draw[18], ϵ_draw[19], ϵ_draw[20]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

           α    0.4851    0.0277     0.0009    0.0033    47.7713    1.0034     ⋯
           β    0.9516    0.0117     0.0004    0.0011   109.5834    0.9991    